<a href="https://colab.research.google.com/github/limshaocong/analyticsEdge/blob/main/Zero_shot_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==3.1.0

     |████████████████████████████████| 884 kB 5.1 MB/s 
     |████████████████████████████████| 3.0 MB 59.9 MB/s 
     |████████████████████████████████| 1.2 MB 58.0 MB/s 
     |████████████████████████████████| 895 kB 57.4 MB/s 


In [2]:
from transformers import pipeline
import pandas as pd

In [3]:
classifier = pipeline("zero-shot-classification", # transfer learning
                      model = "joeddav/xlm-roberta-large-xnli", # multi-lingual model
                      device = 0) # to utilize GPU

Downloading:   0%|          | 0.00/734 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [10]:
import re
from datetime import datetime
import os.path
from os import path
import requests

In [22]:
def analyse(ticker):
  
  file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  
  df = pd.read_csv(file_path)
  
  df['Text_clean'] = df['Text_clean'].astype(str)

  texts = df["Text_clean"]

  chunks = [texts[x : x + 100] for x in range(0, len(texts), 100)] # split into chunks

  results_list = []

  for idx in range(len(chunks)):

    cp1 = datetime.now()
    
    chunk = chunks[idx].tolist()

    candidate_labels = ["positive", "negative", "neutral"]

    hypothesis_template = "The sentiment of this tweet is {}."

    #dicts = classifier(text, candidate_labels, hypothesis_template = hypothesis_template)
    #result = dicts["labels"][0]
    #results_list.append(result)
    
    list_results_dicts = classifier(chunk, candidate_labels, hypothesis_template = hypothesis_template)
    new_list = [i["labels"][0] for i in list_results_dicts]
    results_list.extend(new_list)

    cp2 = datetime.now()

    print(str(idx + 1) + str("/") + str(len(chunks)) + str(" --- ") + str(cp2 - cp1))

  return results_list

In [18]:
ticker_path = "https://raw.githubusercontent.com/limshaocong/analyticsEdge/main/TwitterScraper/Tickers_reduced_full.csv"
tickers = pd.read_csv(ticker_path)
tickers_list = tickers["Tickers"].tolist()
tickers_list = tickers_list[6:]

path = "https://raw.githubusercontent.com/limshaocong/analyticsEdge/main//Datasets/Tweet_Analysis/"

In [ ]:
tickers_list


In [ ]:
for ticker in tickers_list:
  
  file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  
  response = requests.get(file_path)

  if response.status_code < 400:
 
    print(ticker + str(" is present"))

    file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  
    df = pd.read_csv(file_path)

    results_list = analyse(ticker)

    df["Sentiment"] = results_list

    newfilename = ticker + str("_tweets_2020_tagged.csv")

    df.to_csv(newfilename, index = False)
  
  else:

    print(ticker + str(" is missing"))

    continue

AMC is present
1/1495 --- 0:00:02.257753
2/1495 --- 0:00:02.948357
3/1495 --- 0:00:02.853813
4/1495 --- 0:00:02.947352
5/1495 --- 0:00:02.921192
6/1495 --- 0:00:02.198413
7/1495 --- 0:00:03.083459
8/1495 --- 0:00:02.925641
9/1495 --- 0:00:02.123211
10/1495 --- 0:00:03.065571
11/1495 --- 0:00:02.706095
12/1495 --- 0:00:02.216021
13/1495 --- 0:00:02.130700
14/1495 --- 0:00:02.106708
15/1495 --- 0:00:02.296776
16/1495 --- 0:00:02.951051
17/1495 --- 0:00:02.107177
18/1495 --- 0:00:02.676688
19/1495 --- 0:00:02.574420
20/1495 --- 0:00:02.516415
21/1495 --- 0:00:01.811849
22/1495 --- 0:00:02.196485
23/1495 --- 0:00:02.272645
24/1495 --- 0:00:02.247246
25/1495 --- 0:00:01.962966
26/1495 --- 0:00:01.742088
27/1495 --- 0:00:02.752526
28/1495 --- 0:00:02.784717
29/1495 --- 0:00:01.665730
30/1495 --- 0:00:02.845905
31/1495 --- 0:00:03.006568
32/1495 --- 0:00:00.880139
33/1495 --- 0:00:02.848453
34/1495 --- 0:00:02.918170
35/1495 --- 0:00:02.530691
36/1495 --- 0:00:02.865603
37/1495 --- 0:00:01.74